In [1]:
## This file implements neural networks and random forest on p003ppresabs_qual.
## Due to the imbalanced data and the limited cases for class 2, we implement the over-sampling method method.
## For fully-connected neural networks, the accuracy is 97.06% for over-sampling data and 93.63% after regularization.
## For random forest, the accuracy is 98.04% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.53%.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p003ppresabs_qual.csv')
df.shape

(255, 866)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      0
1      0
2      1
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22     0
23     0
24     1
25     0
26     1
27     0
28     0
29     0
      ..
225    0
226    0
227    0
228    1
229    1
230    0
231    0
232    0
233    0
234    0
235    0
236    0
237    0
238    0
239    0
240    0
241    0
242    0
243    0
244    0
245    0
246    0
247    0
248    0
249    0
250    0
251    0
252    0
253    0
254    0
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

0    226
1     26
2      3
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,...,group_8071,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,pheno
0,107,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,109,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,120335,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,120337,1,1,1,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 865)

In [10]:
df_clean.head()

,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATG,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGA,TTTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAA,TTTTTTCTTTTCATAACTGTGTTGGAAATGAATTAAATTAACAGCTCTTTGTGCTTTACGGTGTGTTGC,TTTTTTCAGCATTGTCTACATTACTTAACATTCGTGTTTGTAAGTAATATTGACCGCCAATATTTAGACACTTTATAAGTATGCCATTCATCATTTTTAA,TTTTTTATCTCACCAATTTTGTAATACATCGTTCTCGTCCTCCTTGTCTTCTTCGTCCTCCTCGTTATCTTCTTCGTTTTGTAATTCATAAATTTTGTTT,TTTTTTAGTTTATCCAATGATTGATGTTATAATAATACTAAATTTGTATCTATAAAAAAGTAATGAGCATTTGTGCGCATATGATGATGTAAAGCGTAAA,TTTTTTAGGTACC,TTTTTGCATTCA,TTTTTCTTATTTTA,...,group_8071,group_8208,group_8664,group_8665,group_8666,group_8667,group_8836,group_8913,group_9508,pheno
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1
3,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 864) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 226), (1, 226), (2, 226)]


Using TensorFlow backend.


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
############# Fully-Connected Neural Network ################

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [16]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [17]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 474 samples, validate on 204 samples
Epoch 1/100
474/474 [==============================] - 0s 458us/step - loss: 0.8779 - accuracy: 0.5992 - val_loss: 0.5950 - val_accuracy: 0.7892
Epoch 2/100
474/474 [==============================] - 0s 92us/step - loss: 0.5112 - accuracy: 0.8228 - val_loss: 0.4360 - val_accuracy: 0.8382
Epoch 3/100
474/474 [==============================] - 0s 77us/step - loss: 0.3893 - accuracy: 0.8523 - val_loss: 0.3566 - val_accuracy: 0.8873
Epoch 4/100
474/474 [==============================] - 0s 80us/step - loss: 0.3089 - accuracy: 0.9008 - val_loss: 0.3085 - val_accuracy: 0.8922
Epoch 5/100
474/474 [==============================] - 0s 88us/step - loss: 0.2606 - accuracy: 0.9304 - val_loss: 0.2681 - val_accuracy: 0.8971
Epoch 6/100
474/474 [==============================] - 0s 92us/step - loss: 0.2483 - accuracy: 0.9030 - val_loss: 0.2553 - val_accuracy: 0.9265
Epoch 7/100
474/474 [==============================] - 0s 89us/step - loss: 0.2114 - accu

474/474 [==============================] - 0s 149us/step - loss: 0.0076 - accuracy: 1.0000 - val_loss: 0.1308 - val_accuracy: 0.9657
Epoch 58/100
474/474 [==============================] - 0s 129us/step - loss: 0.0073 - accuracy: 1.0000 - val_loss: 0.1162 - val_accuracy: 0.9755
Epoch 59/100
474/474 [==============================] - 0s 107us/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.1115 - val_accuracy: 0.9755
Epoch 60/100
474/474 [==============================] - 0s 124us/step - loss: 0.0069 - accuracy: 1.0000 - val_loss: 0.1178 - val_accuracy: 0.9755
Epoch 61/100
474/474 [==============================] - 0s 319us/step - loss: 0.0070 - accuracy: 1.0000 - val_loss: 0.1062 - val_accuracy: 0.9755
Epoch 62/100
474/474 [==============================] - 0s 98us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.1085 - val_accuracy: 0.9755
Epoch 63/100
474/474 [==============================] - 0s 107us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.1231 - val_accuracy:

In [19]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

204/204 [==============================] - 0s 174us/step
over-sampling test accuracy: 97.06%


In [20]:
#### add regularizor and dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [21]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 474 samples, validate on 204 samples
Epoch 1/100
474/474 [==============================] - 0s 560us/step - loss: 4.3837 - accuracy: 0.5021 - val_loss: 0.8017 - val_accuracy: 0.7108
Epoch 2/100
474/474 [==============================] - 0s 128us/step - loss: 3.4632 - accuracy: 0.6329 - val_loss: 0.7418 - val_accuracy: 0.8431
Epoch 3/100
474/474 [==============================] - 0s 123us/step - loss: 3.3826 - accuracy: 0.6751 - val_loss: 0.7146 - val_accuracy: 0.8480
Epoch 4/100
474/474 [==============================] - 0s 121us/step - loss: 2.8963 - accuracy: 0.7046 - val_loss: 0.7423 - val_accuracy: 0.8333
Epoch 5/100
474/474 [==============================] - 0s 108us/step - loss: 3.1340 - accuracy: 0.6730 - val_loss: 0.8337 - val_accuracy: 0.8529
Epoch 6/100
474/474 [==============================] - 0s 106us/step - loss: 2.8961 - accuracy: 0.6835 - val_loss: 0.7843 - val_accuracy: 0.8627
Epoch 7/100
474/474 [==============================] - 0s 290us/step - loss: 2.8727 

474/474 [==============================] - 0s 158us/step - loss: 1.1497 - accuracy: 0.8017 - val_loss: 0.8088 - val_accuracy: 0.9314
Epoch 58/100
474/474 [==============================] - 0s 114us/step - loss: 1.0617 - accuracy: 0.8333 - val_loss: 0.7246 - val_accuracy: 0.9363
Epoch 59/100
474/474 [==============================] - 0s 85us/step - loss: 1.1160 - accuracy: 0.8101 - val_loss: 0.8254 - val_accuracy: 0.9314
Epoch 60/100
474/474 [==============================] - 0s 79us/step - loss: 1.1297 - accuracy: 0.8143 - val_loss: 0.8632 - val_accuracy: 0.9314
Epoch 61/100
474/474 [==============================] - 0s 101us/step - loss: 1.1104 - accuracy: 0.7975 - val_loss: 0.8632 - val_accuracy: 0.9265
Epoch 62/100
474/474 [==============================] - 0s 120us/step - loss: 1.2191 - accuracy: 0.7911 - val_loss: 0.8100 - val_accuracy: 0.9412
Epoch 63/100
474/474 [==============================] - 0s 116us/step - loss: 1.3499 - accuracy: 0.7827 - val_loss: 0.7705 - val_accuracy: 

In [23]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

204/204 [==============================] - 0s 121us/step
over-sampling test accuracy: 93.14%


In [24]:
############## Random Forest ##############

In [25]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 98.04%


In [28]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.97916667 1.         0.96875    1.         0.97849462]
0.985282258064516
